In [3]:
from pathlib import Path

In [4]:
sample = Path('crop')
labels = Path('result/labels')

In [7]:
str(Path.cwd())

'/home/astra/projects/hackathon/omsk_chempionat/model-predict'

In [17]:
!python yolov5/detect.py --weights ../best.pt --source crop/ --imgsz 256 256 --save-txt --project result/

detect: weights=['../best.pt'], source=crop/, data=yolov5/data/coco128.yaml, imgsz=[256, 256], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=result/, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-203-g6371de8 Python-3.10.0 torch-1.12.1+cu102 CPU

Fusing layers... 
[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Traceback (most recent call last):
  File "/home/astra/projects/hackathon/omsk_chempionat/model-predict/yolov5/detect.py", line 258, in <module>
    main(opt)
  File "/home/astra/projects/hackathon/omsk_chempionat/model-predict/yolov5/detect.py", line 253, in main
    run(**vars(opt))
  File "/home/astra/projects/hackathon

In [ ]:
def get_files(files_dir: Path, suff: str = 'png') -> List[Path]:
    return sorted(Path(files_dir).glob(f'*.{suff}'))

In [ ]:
labels_list = get_files(labels)

In [ ]:
def str_to_float(labels_list: list[list[str]]) -> list[list[float]]:
    float_labels = []
    for label in labels_list:
        int_label = [float(x) for x in label]
        float_labels.append(int_label)
    return float_labels

In [ ]:
def read_txt(label_file: Path) -> list[list[float]]:
    # функция возвращает список боксов с координатами в формате
    # [номер класса объекта] [x - центр] [y -центр] [ширина от центра] [высота от центра]
    # значения координат в диапазоне от 0 до 1 (доля от общих рамзеров картинки)
    with open(label_file, 'r') as labels:
        labels_list = labels.readlines()
        labels_list = [row.replace('\n', '') for row in labels_list]
        labels_list = [row.split(' ') for row in labels_list]
        return str_to_float(labels_list)
        

In [ ]:
def fraction_to_pixel(labels: list[list[float]], size: tuple[int, int] = (256,256), x: int, y: int) -> list[list[int]]:
    width, height = size
    width = width + x
    height = height + y
    _, x_center, y_center, obj_width, obj_height = labels
    x_center = x_center * width
    obj_width = obj_width * width
    y_center = y_center * height
    obj_height = obj_height * height
    radius = int((obj_width + obj_height) / 2)
    return [x_center, y_center, radius]

In [ ]:
def get_x_y(label_file: Path) -> tuple[int, int]:
    # .x.%03d.y.%03d.jpg
    name = label_file.stem
    name_list = name.split('.')
    x_min = name_list[2]
    y_min = name_list[4]
    return x_min, y_min

In [ ]:
for label_file in labels_list:
    labels = read_txt(label_file)
    x_min, y_min = get_x_y(label_file)
    pixel_labels = fraction_to_pixel(labels, x=x_min, y=y_min)
    break